In [1]:
from datasets import Dataset, concatenate_datasets
from docx import Document
import json
import re


/home/ysx/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 读取docx文档
def read_docx_file(file_path):
    document = Document(file_path)
    text = ""
    for paragraph in document.paragraphs:
        text += paragraph.text + "\n"
    return text

# 从文本中提取json部分
def extract_json(text):
    # 使用正则表达式匹配有效的JSON部分
    json_pattern = r'\{.*?\}'
    matches = re.findall(json_pattern, text, re.DOTALL)
    print(len(matches))
    
    if len(matches) > 0:
        return matches
    else:
        return None
    

# 把字符串转换成json格式
def list_to_json(list):
    r = []
    for t in list:
        j = json.loads(t)
        r.append(j)
    return r



In [3]:
# 读取docx文档
file_path = "../data/docx/zysft.docx"
text = read_docx_file(file_path)

# 提取json部分
json_text = extract_json(text)

# 转换成json list
list = list_to_json(json_text)

#转换成datasets
dataset = Dataset.from_list(list)

dataset = dataset.rename_column('instruct', 'instruction')


#保存数据
dataset.save_to_disk("../data/datasets/zysft")

100


Saving the dataset (1/1 shards): 100%|██████████| 100/100 [00:00<00:00, 18687.86 examples/s]


In [9]:

j = dataset.from_json("../data/json/alpaca_data_cleaned_archive.json")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 826.95it/s]
Generating train split: 51759 examples [00:00, 141129.84 examples/s]


In [10]:
j

Dataset({
    features: ['instruction', 'output', 'input'],
    num_rows: 51759
})

In [13]:
combined_dataset = concatenate_datasets([dataset, j])

In [15]:
combined_dataset.save_to_disk("../data/datasets/zysft")

Saving the dataset (1/1 shards): 100%|██████████| 51859/51859 [00:00<00:00, 625033.87 examples/s]


In [19]:
combined_dataset[1000]

{'instruction': 'Generate a story describing the emotions of a character in a given situation.',
 'input': "Alice is stuck in traffic, trying to get to her parent's house for Thanksgiving.",
 'output': "Alice pounded her fists on the steering wheel as she looked out the window at the gridlocked cars stretching on for miles. She was desperate to reach her parents' house in time for Thanksgiving, but it seemed like every driver on the road had decided to take the same route she had. Anger and frustration built within her as the minutes ticked by, feeling like hours. Her only hope was that somehow, some way, she would still make it on time."}

In [4]:
dataset[0]

{'instruction': '定义五行的生克制化规律。',
 'input': '五行的生克制化规律是五行结构系统在正常情况下的自动调节机制。五行之间通过相生相克的关系，维持事物的平衡和发展。相生是指五行之间互相滋生和促进的关系，如木生火、火生土等。相克是指五行之间相互制约的关系，如木克土、土克水等。制化关系是生克关系的结合，生中有克、克中有生，相反相成，维持和促进事物相对平衡协调和发展变化。',
 'output': '五行的生克制化规律是指五行相生和相克的动态平衡，其中相生规律体现为五行之间的相互滋养和促进，相克规律体现为五行之间的相互制约和克制，制化规律则是生克结合的复杂关系，确保事物的平衡发展。'}